# 제주 HVDC 연계선 수전 전력 분석

## 프로젝트 목표
- 제주 HVDC 수전 전력의 시간대 패턴과 일별 변동성을 시각화
- 계통 제약과 출력 제한 이슈를 데이터 기반으로 설명

## 핵심 질문
1. 하루 동안 HVDC 수전 전력은 어떤 시간대 패턴을 가지는가?
2. 날짜별로 이 패턴은 얼마나 변동하는가?
3. 이 변동은 계통 제약과 출력 제한 논의와 어떻게 연결되는가?

## 데이터
- HVDC#1: 제주-해남 1연계선 (±180kV, 300MW)
- HVDC#2: 제주-해남 2연계선 (±250kV, 400MW)
- HVDC#3: 제주-완도 3연계선 (±200kV, 200MW)

---
## 1. 환경 설정 및 데이터 로드

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 시각화 스타일 설정
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# 데이터 파일 경로
DATA_FILES = {
    'HVDC#1': 'data/제주 일별 시간대별 연계선 수전전력(HVDC#1)_251103.csv',
    'HVDC#2': 'data/제주 일별 시간대별 연계선 수전전력(HVDC#2)_251103.csv',
    'HVDC#3': 'data/제주 일별 시간대별 연계선 수전전력(HVDC#3)_251103.csv'
}

def load_hvdc_data(filepath, name):
    """HVDC 데이터 로드 및 전처리"""
    df = pd.read_csv(filepath, encoding='euc-kr')
    
    # 첫 번째 컬럼이 날짜
    date_col = df.columns[0]
    df = df.rename(columns={date_col: 'date'})
    
    # wide -> long 형식 변환
    hour_cols = [col for col in df.columns if col != 'date']
    df_long = df.melt(id_vars=['date'], value_vars=hour_cols, 
                      var_name='hour_raw', value_name='power')
    
    # 시간 파싱 (1시, 2시, ... -> 1, 2, ...)
    df_long['hour'] = df_long['hour_raw'].str.replace('시', '').astype(int)
    df_long['date'] = pd.to_datetime(df_long['date'])
    df_long['hvdc'] = name
    
    return df_long[['date', 'hour', 'power', 'hvdc']]

# 세 파일 모두 로드
dfs = []
for name, path in DATA_FILES.items():
    df = load_hvdc_data(path, name)
    dfs.append(df)
    print(f"{name}: {len(df):,}개 데이터 로드")

df_all = pd.concat(dfs, ignore_index=True)
print(f"\n전체 데이터: {len(df_all):,}개")

In [ ]:
# 세 연계선 합산 데이터 생성
df_total = df_all.groupby(['date', 'hour'])['power'].sum().reset_index()
df_total['hvdc'] = 'Total'

print(f"분석 기간: {df_total['date'].min().date()} ~ {df_total['date'].max().date()}")
print(f"총 일수: {df_total['date'].nunique()}일")

df_total.head()

---
## 2. 시간대별 수전 전력 패턴 분석

In [ ]:
# 시간대별 평균 수전 전력 (전체 합산)
hourly_avg = df_total.groupby('hour')['power'].mean()

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(hourly_avg.index, hourly_avg.values, color='steelblue', alpha=0.8, edgecolor='navy')
ax.plot(hourly_avg.index, hourly_avg.values, 'o-', color='darkred', linewidth=2, markersize=6)

# 평균선
ax.axhline(y=hourly_avg.mean(), color='red', linestyle='--', linewidth=1.5, 
           label=f'전체 평균: {hourly_avg.mean():.1f} MW')

ax.set_xlabel('시간대', fontsize=12)
ax.set_ylabel('평균 수전 전력 (MW)', fontsize=12)
ax.set_title('시간대별 평균 HVDC 총 수전 전력 (1+2+3호기)', fontsize=14, fontweight='bold')
ax.set_xticks(range(1, 25))
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# 연계선별 시간대 패턴 비교
fig, ax = plt.subplots(figsize=(12, 6))

colors = {'HVDC#1': '#1f77b4', 'HVDC#2': '#ff7f0e', 'HVDC#3': '#2ca02c'}

for hvdc_name in ['HVDC#1', 'HVDC#2', 'HVDC#3']:
    hvdc_data = df_all[df_all['hvdc'] == hvdc_name]
    hourly = hvdc_data.groupby('hour')['power'].mean()
    ax.plot(hourly.index, hourly.values, 'o-', linewidth=2, markersize=5, 
            label=hvdc_name, color=colors[hvdc_name])

ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.set_xlabel('시간대', fontsize=12)
ax.set_ylabel('평균 수전 전력 (MW)', fontsize=12)
ax.set_title('연계선별 시간대 평균 수전 전력', fontsize=14, fontweight='bold')
ax.set_xticks(range(1, 25))
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("※ 음수 = 제주 → 육지 역송 (제주 발전량 > 수요)")

---
## 3. 날짜별 시간대 패턴 비교 (다중 선 그래프)

In [ ]:
# 최근 N일 데이터 선택
N_DAYS = 7
recent_dates = sorted(df_total['date'].unique())[-N_DAYS:]

fig, ax = plt.subplots(figsize=(14, 7))

cmap = plt.cm.viridis
for i, date in enumerate(recent_dates):
    daily_data = df_total[df_total['date'] == date].sort_values('hour')
    color = cmap(i / len(recent_dates))
    ax.plot(daily_data['hour'], daily_data['power'], 
            marker='o', markersize=4, alpha=0.8, linewidth=1.5,
            label=str(date.date()), color=color)

ax.set_xlabel('시간대', fontsize=12)
ax.set_ylabel('수전 전력 (MW)', fontsize=12)
ax.set_title(f'최근 {N_DAYS}일간 시간대별 HVDC 총 수전 전력', fontsize=14, fontweight='bold')
ax.set_xticks(range(1, 25))
ax.legend(title='날짜', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 4. 일별 총 수전 전력 요약

In [ ]:
# 일별 총 수전 전력 계산
daily_total = df_total.groupby('date')['power'].sum().reset_index()
daily_total.columns = ['date', 'total_power']

fig, ax = plt.subplots(figsize=(14, 6))

# 막대 색상: 평균 이하면 주황색
avg_power = daily_total['total_power'].mean()
colors = ['steelblue' if p >= avg_power else 'coral' for p in daily_total['total_power']]

ax.bar(range(len(daily_total)), daily_total['total_power'], color=colors, alpha=0.8)
ax.axhline(y=avg_power, color='red', linestyle='--', linewidth=2, 
           label=f'평균: {avg_power:,.0f} MWh')

ax.set_xlabel('날짜', fontsize=12)
ax.set_ylabel('일간 총 수전 전력 (MWh)', fontsize=12)
ax.set_title('일별 HVDC 총 수전 전력', fontsize=14, fontweight='bold')

# x축 라벨
n_labels = min(15, len(daily_total))
step = max(1, len(daily_total) // n_labels)
ax.set_xticks(range(0, len(daily_total), step))
ax.set_xticklabels([d.strftime('%m-%d') for d in daily_total['date'][::step]], 
                   rotation=45, ha='right')

ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---
## 5. 변동성 분석

In [ ]:
# 시간대별 변동성 (표준편차)
hourly_stats = df_total.groupby('hour')['power'].agg(['mean', 'std', 'min', 'max'])

fig, ax = plt.subplots(figsize=(12, 6))

ax.fill_between(hourly_stats.index, 
                hourly_stats['mean'] - hourly_stats['std'],
                hourly_stats['mean'] + hourly_stats['std'],
                alpha=0.3, color='blue', label='평균 ± 1 표준편차')
ax.plot(hourly_stats.index, hourly_stats['mean'], 'b-', linewidth=2, marker='o', label='평균')
ax.plot(hourly_stats.index, hourly_stats['min'], 'g--', linewidth=1, alpha=0.7, label='최소')
ax.plot(hourly_stats.index, hourly_stats['max'], 'r--', linewidth=1, alpha=0.7, label='최대')

ax.set_xlabel('시간대', fontsize=12)
ax.set_ylabel('수전 전력 (MW)', fontsize=12)
ax.set_title('시간대별 수전 전력 변동 범위', fontsize=14, fontweight='bold')
ax.set_xticks(range(1, 25))
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 역송(음수) 발생 빈도 분석
df_all['is_reverse'] = df_all['power'] < 0
reverse_by_hour = df_all.groupby(['hour', 'hvdc'])['is_reverse'].mean() * 100

reverse_pivot = reverse_by_hour.unstack()

fig, ax = plt.subplots(figsize=(12, 6))
reverse_pivot.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)

ax.set_xlabel('시간대', fontsize=12)
ax.set_ylabel('역송 발생 비율 (%)', fontsize=12)
ax.set_title('시간대별 역송(제주→육지) 발생 비율', fontsize=14, fontweight='bold')
ax.set_xticklabels(range(1, 25), rotation=0)
ax.legend(title='연계선')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("※ 역송 비율이 높은 시간대 = 신재생 발전량 > 수요, 출력제한 가능성 높음")

---
## 6. 결론 및 해석

In [ ]:
# 주요 통계 요약
print("=" * 60)
print("제주 HVDC 수전 전력 분석 요약")
print("=" * 60)

# 분석 기간
print(f"\n분석 기간: {df_total['date'].min().date()} ~ {df_total['date'].max().date()}")

# 피크 시간대
peak_hour = hourly_avg.idxmax()
min_hour = hourly_avg.idxmin()
print(f"\n1. 시간대 패턴")
print(f"   - 수전 전력 최대 시간대: {peak_hour}시 ({hourly_avg[peak_hour]:.1f} MW)")
print(f"   - 수전 전력 최소 시간대: {min_hour}시 ({hourly_avg[min_hour]:.1f} MW)")
print(f"   - 최대/최소 비율: {hourly_avg[peak_hour] / hourly_avg[min_hour]:.1f}배")

# 일별 변동
print(f"\n2. 일별 변동성")
print(f"   - 일평균 총 수전: {daily_total['total_power'].mean():,.0f} MWh")
print(f"   - 표준편차: {daily_total['total_power'].std():,.0f} MWh")
print(f"   - 변동계수(CV): {daily_total['total_power'].std() / daily_total['total_power'].mean() * 100:.1f}%")

# 역송 분석
reverse_rate = (df_all['power'] < 0).mean() * 100
reverse_hours = hourly_stats[hourly_stats['min'] < 0].index.tolist()
print(f"\n3. 역송(제주→육지) 현황")
print(f"   - 전체 역송 발생 비율: {reverse_rate:.1f}%")
print(f"   - 역송 발생 시간대: {reverse_hours if reverse_hours else '없음'}")

# 계통 제약 시사점
print(f"\n4. 계통 제약 시사점")
low_demand_hours = hourly_stats[hourly_stats['mean'] < hourly_avg.mean() * 0.7].index.tolist()
if low_demand_hours:
    print(f"   - 수전 저조 시간대 (평균 70% 미만): {low_demand_hours}")
    print(f"   - 이 시간대 신재생 발전 집중 시 출력제한 가능성 존재")

---
## 결론 요약

### 분석 결과

1. **시간대 패턴**: 저녁 피크(18-20시) 시간대에 수전 전력이 최대, 낮 시간대(11-14시)에 최소

2. **일별 변동성**: 일별 총 수전 전력의 변동계수(CV)를 통해 날씨/계절에 따른 변동 확인

3. **역송 현상**: HVDC#3에서 낮 시간대 음수값 발생 → 제주 신재생 발전량 초과 시 육지로 역송

4. **출력제한 추론**: 낮 시간대 수전 전력 급감 + 역송 발생 = 신재생 출력제한 가능성

5. **시사점**: 제주 계통의 수용 한계와 HVDC 연계선 활용 패턴이 출력제한의 구조적 원인을 설명